In [5]:
import os
import pandas as pd
import cv2
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [23]:
def preprocess_images_from_folder(folder): 
    images = []
    for filename in tqdm(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None: 
            
            # 1. Dilate the image, to get rid of the text
            dilated_img = cv2.dilate(img, np.ones((32,32), np.uint8))
            
            # 2. Median blur the result with a decent sized kernel to further suppress any text.
            bg_img = cv2.medianBlur(dilated_img, 23)

            # 3. Calculate the difference between the original and the background we just obtained. The bits that are identical will be black (close to 0 difference), the text will be white (large difference).
            diff_img = 255 - cv2.absdiff(img, bg_img)

            # 4. Normalize the image, so that we use the full dynamic range.
            norm_img = diff_img.copy() # Needed for 3.x compatibility
            end_one_img = cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)

            # 5. Truncate the remaining gray away and re-normalize the image
            _, thr_img = cv2.threshold(norm_img, 255, 0, cv2.THRESH_TRUNC)
            end_img = cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
            
            resized = image_resize(end_img, width=1024)
            
            new_image = np.zeros(resized.shape, resized.dtype)
            alpha = 1.0 # Simple contrast control
            beta = 5    # Simple brightness control

            new_image = cv2.convertScaleAbs(resized, alpha=alpha, beta=beta)
            
            colorized_img = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
            
            result = cv2.imwrite('../processed_data/watercolor/' + filename, colorized_img)

folder = '../data/Watercolor photos'

images = preprocess_images_from_folder(folder)

100%|██████████| 349/349 [05:58<00:00,  1.03s/it]
